In [34]:
import pandas as pd
import osgb

In [35]:
df = pd.read_csv('coords.csv')
df.head()

,Unnamed: 0,Grid Ref,Location,Question,Answer
0,1,SP 9452 1567,Pitstone Windmill,How many steps up to the windmill?,NaN
1,2,SP 9589 1534,FPS at X tracks,How far to Ivinghoe Beacon in km?,NaN
2,3,SP 9596 1687,Base of Trig-point on Ivinghoe Beacon,List the characters on metal plate?,NaN
3,4,SP 9685 1687,The HG below Gallows Hill,Who donated the gate in 2011?,NaN
4,5,SP 9700 1582,FPS before entering pine trees,What Trail is it?,NaN


In [36]:
#grid = osgb.parse_grid("SP 9644 1243")
#lonlat = osgb.grid_to_ll(grid)

In [44]:
df['lat_long']=df['Grid Ref'].apply(osgb.parse_grid).apply(osgb.grid_to_ll)
df.head()

,Unnamed: 0,Grid Ref,Location,Question,Answer,lat_long
0,1,SP 9452 1567,Pitstone Windmill,How many steps up to the windmill?,NaN,"(51.831665, -0.629699)"
1,2,SP 9589 1534,FPS at X tracks,How far to Ivinghoe Beacon in km?,NaN,"(51.828466, -0.609916)"
2,3,SP 9596 1687,Base of Trig-point on Ivinghoe Beacon,List the characters on metal plate?,NaN,"(51.842206, -0.608476)"
3,4,SP 9685 1687,The HG below Gallows Hill,Who donated the gate in 2011?,NaN,"(51.842053, -0.595561)"
4,5,SP 9700 1582,FPS before entering pine trees,What Trail is it?,NaN,"(51.832589, -0.593679)"
